<a href="https://colab.research.google.com/github/aiguozhe01/DIC_Assignment/blob/master/Sprint_13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sprint ディープラーニングフレームワーク1**

# 【問題1】スクラッチを振り返る

* ここまでのスクラッチを振り返り、ディープラーニングを実装するためにはどのようなものが必要だったかを列挙しろ。

**【解答】**

1. 入力データの準備
2. テストデータと訓練データに分ける。
3. 入力データ内から、ランダムに「ミニバッチ」を取り出す
4. 「バイアス」と「重み」を初期化する。
5. 順伝搬を行う
    1. 訓練データをパッド分に展開する。
    2. 訓練データをフィルター（重み）に通して乗算する。
    3. バイアスを加算する。
    4. 活性化関数を行う
    5. 全層を通過後、出力の際softmax関数を用いて、値を正規化する。
    6. 出力した値を記録して、次のミニバッチの学習を行う。
6. 逆伝搬を行う
    1. 各重みパラメータに関する損失関数の勾配を求める

7. 重みパラメータを勾配方向に微小量だけ更新する。

In [12]:
!python3 -m pip install tensorflow==2.3.0

     |████████████████████████████████| 320.4MB 50kB/s 
     |████████████████████████████████| 460kB 41.9MB/s 
     |████████████████████████████████| 6.8MB 44.2MB/s 
  Found existing installation: tensorflow-estimator 1.14.0
    Uninstalling tensorflow-estimator-1.14.0:
      Successfully uninstalled tensorflow-estimator-1.14.0
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


# 【問題2】スクラッチとTensorFlowの対応を考える

In [5]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [ ]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを2値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="datasets_19_420_Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df = df[(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y[y=='Iris-versicolor'] = 0
y[y=='Iris-virginica'] = 1
# print(y, y.shape, type(y))
y = y.astype(np.int)[:, np.newaxis]
# print(y, y.shape, type(y))
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 20
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)    
        total_loss = np.empty(total_batch)
        # total_loss = 0
        total_acc = np.empty(total_batch)
        # total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= total_loss.mean()
        #total_loss /= batch_size
        total_acc /= total_acc.mean()
        #total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.8f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 7.44431210, val_loss : 12.3300, acc : 0.750, val_acc : 0.375
Epoch 1, loss : 4.21767378, val_loss : 4.9650, acc : 0.750, val_acc : 0.625
Epoch 2, loss : 13.51782227, val_loss : 3.4740, acc : 0.500, val_acc : 0.812
Epoch 3, loss : 4.26804495, val_loss : 5.2629, acc : 0.750, val_acc : 0.562
Epoch 4, loss : 5.67510748, val_loss : 2.3903, acc : 0.500, val_acc : 0.750
Epoch 5, loss : 0.00879623, val_loss : 0.7658, acc : 1.000, val_acc : 0.875
Epoch 6, loss : 0.00003815, val_loss : 0.0002, acc : 1.000, val_acc : 1.000
Epoch 7, loss : 0.00000000, val_loss : 0.0039, acc : 1.000, val_acc : 1.000
Epoch 8, loss : 0.00000000, val_loss : 0.0001, acc : 1.000, val_acc : 1.000
Epoch 9, loss : 0.00000000, val_loss : 0.0013, acc : 1.000, val_acc : 1.000
Epoch 10, loss : 0.00000000, val_loss : 0.0003, acc : 1.000, val_acc : 1.000
Epoch 11, loss : 0.00000000, val_loss : 0.0185, acc : 1.000, val_acc : 1.000
Epoch 12, loss : 0.00000000, val_loss : 0.1414, acc : 1.000, val_acc : 0.938
Epoch 1

**【疑似コード】**

**Comparision Between Scratch and TensorFlow(v1.15)**

1. データセットの読込
2. データフレームから条件の抽出
3. データフレームから目的変数の抽出
4. データフレームから説明変数の抽出
5. pandas.dataframeからnumpy.arrayへ変換
6. 目的変数を数値に変換
7. 目的変数のdtypeをint型に変換して、2次元配列に再成形
8. 説明変数と目的変数を訓練データと試験データに分別する。
9. 分別した訓練データ（説明変数、目的変数）をさらに2分別する。
----
10. ランダムに10個データを一纏まりとしたミニバッチを行う関数を作成
----
11. ハイパーパラメータの設定
    1. 学習率
    2. バッチサイズ
    3. エポック数
    4. 第1隠れ層のノード数
    5. 第2隠れ層のノード数
    6. 入力データの列数（特徴数）
    7. 入力データの行数（データ数）
    8. クラス数
12. **計算グラフに渡す引数の形を決める**
13. ミニバッチ関数を用い、訓練データ用のバッチを作成してget_mini_batch_trainに格納
----
14. **3層ニューラルネットワークの順伝搬する関数を作成**
    1. 工程11で設定したハイパーパラメータを用いて、重み3層とバイアス3つを作成
    2. 1～2層目：入力データと重みの内積にバイアスを加算した後、活性化関数（relu）を適用
    3. 出力層：2層目の出力と3つ目の重みの内積に3つ目のバイアスを加算して終了
15. **ネットワーク構造の読込**
    * 入力データが引数
16. **目的関数：目的変数に損失関数を適用**
    * シグモイド関数、交差エントロピー誤差を経て、数値の平均値を返す。
17. **最適化手法を定義**
    * Adam最適化法を指定の学習率でインスタンス化
    * インスタンス化した目的関数を最小化し、最適化して返す。
18. **推定結果の設定**
    * tf.signを用いて目的変数とネットワーク構造化した後、シグモイド関数を適用した説明変数の符号が同一かを判定
    * booliang型を返す
19. **指標値計算の設定**
    * シグモイド済みの推定結果（booliang型）をfloat32に変換して、平均値を返し、それを指標値とする。
20. **variableの初期化**
----
21. **計算グラフの実行**
    1. TensorFlow 1.x 特有のコマンドtf.Sessionを用いて計算グラフを実行する。
    2. sess.run(init): 数値の初期化
    3. 指定したエポック数の下、for文をループさせる。
        1. バッチ合計 = 訓練データの行数分を指定バッチサイズで除算し、数値を切り上げし、intに変換する。
        2. total_loss, total_accをゼロに初期化
        3. 新たなfor文：get_mini_batch_trainに格納した要素とインデックスを同時に取得
        4. 複数のイテラブル（変数名）mini_batch_x, mini_batch_yに要素取得して、ディクショナリー化
        5. 上記の工程4の値と、最適化した損失関数と指標値を用いsess.runで実行
        6. 損失と指標値を加算代入する。
     4. 加算代入を終えたtotal_loss, total_accをサンプル数（データフレームの行数）で除算し、複合代入させる。
     5. val_loss, val_accを算出するため、sess.run関数を実行（中身は最適化した損失関数と指標値、ディクショナリー化したX_val、y_val）
     6. 各エポック毎の損失、val損失、指標値、val指標値を表示
     7. 試験データと指標値で同じ計算グラフ廻して、評価値を出す。

# 【問題3】3種類全ての目的変数を使用したIrisのモデルを作成

In [ ]:
"""
TensorFlowで実装したニューラルネットワークを使いIrisデータセットを3値分類する
"""
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import tensorflow as tf
# データセットの読み込み
dataset_path ="datasets_19_420_Iris.csv"
df = pd.read_csv(dataset_path)
# データフレームから条件抽出
df =  df[(df["Species"] == "Iris-setosa")|(df["Species"] == "Iris-versicolor")|(df["Species"] == "Iris-virginica")]
# y = df["Species"]
X = df.loc[:, ["SepalLengthCm", "SepalWidthCm", "PetalLengthCm", "PetalWidthCm"]]
y = np.array(y)
X = np.array(X)
# ラベルを数値に変換
y = pd.get_dummies(df.Species, prefix='Species')
# y[y=='Iris-versicolor'] = 0
# y[y=='Iris-virginica'] = 1
# y = y.astype(np.int)[:, np.newaxis]
y = y.to_numpy()
# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)
class GetMiniBatch:
    """
    ミニバッチを取得するイテレータ

    Parameters
    ----------
    X : 次の形のndarray, shape (n_samples, n_features)
      訓練データ
    y : 次の形のndarray, shape (n_samples, 1)
      正解値
    batch_size : int
      バッチサイズ
    seed : int
      NumPyの乱数のシード
    """
    def __init__(self, X, y, batch_size = 10, seed=0):
        self.batch_size = batch_size
        np.random.seed(seed)
        shuffle_index = np.random.permutation(np.arange(X.shape[0]))
        self.X = X[shuffle_index]
        self.y = y[shuffle_index]
        self._stop = np.ceil(X.shape[0]/self.batch_size).astype(np.int)
    def __len__(self):
        return self._stop
    def __getitem__(self,item):
        p0 = item*self.batch_size
        p1 = item*self.batch_size + self.batch_size
        return self.X[p0:p1], self.y[p0:p1]        
    def __iter__(self):
        self._counter = 0
        return self
    def __next__(self):
        if self._counter >= self._stop:
            raise StopIteration()
        p0 = self._counter*self.batch_size
        p1 = self._counter*self.batch_size + self.batch_size
        self._counter += 1
        return self.X[p0:p1], self.y[p0:p1]
# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 50
num_epochs = 100
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 3
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
# loss_op = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=logits))
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
correct_pred = tf.equal(tf.argmax(Y, 1), tf.argmax(tf.nn.softmax(logits), 1))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)    
        total_loss = np.empty(total_batch)
        # total_loss = 0
        total_acc = np.empty(total_batch)
        # total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= total_loss.mean()
        #total_loss /= batch_size
        total_acc /= total_acc.mean()
        #total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.8f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))

Epoch 0, loss : 126.65926361, val_loss : 123.8705, acc : 0.348, val_acc : 0.292
Epoch 1, loss : 83.51921082, val_loss : 78.2784, acc : 0.239, val_acc : 0.375
Epoch 2, loss : 33.43477249, val_loss : 33.5225, acc : 0.283, val_acc : 0.292
Epoch 3, loss : 8.12497616, val_loss : 10.3206, acc : 0.630, val_acc : 0.625
Epoch 4, loss : 22.82771301, val_loss : 20.3210, acc : 0.391, val_acc : 0.333
Epoch 5, loss : 3.18047500, val_loss : 3.0254, acc : 0.652, val_acc : 0.708
Epoch 6, loss : 0.71592098, val_loss : 1.4878, acc : 0.913, val_acc : 0.750
Epoch 7, loss : 2.91247964, val_loss : 4.1004, acc : 0.717, val_acc : 0.583
Epoch 8, loss : 2.74655461, val_loss : 4.9936, acc : 0.761, val_acc : 0.667
Epoch 9, loss : 2.02749372, val_loss : 4.8485, acc : 0.826, val_acc : 0.667
Epoch 10, loss : 0.67740995, val_loss : 2.3692, acc : 0.935, val_acc : 0.833
Epoch 11, loss : 0.66403735, val_loss : 1.1853, acc : 0.870, val_acc : 0.833
Epoch 12, loss : 0.39142495, val_loss : 1.1985, acc : 0.957, val_acc : 0.79

## 変更箇所

* 13行目：Iris-setosaをdf["Species"]に追加
* 19行目：3値分類のため、on-hot encodingに対応するためpandasのget_dummies()を用いる
    * よって、20~22行目までが不要となる。
* 23行目：get_dummies()したため、yのデータ型式をpandas.coreからnumpyに戻す。
    * to_numpy()を使用
    * 16行目のy = np.array(y)でも同様の効果
* 74行目：77行目のYに格納するplaceholderの列数を1から3に変えるため、n_classes = 3にする。
* 105行目：sigmoid_cross_entropyからsoftmax_corss_entropyに変更
* 111行目：多値用にtf.argmaxを使用。（複数の割合から最も高い値を返す）

# 【問題4】House Pricesのモデルを作成

* 目的変数：SalePrice
* 説明変数：GrLivArea, YearBuilt
* 分類問題ではなく、回帰問題（目的変数を予測する）
* DNNRegressor with Reluを用いる。
* 隠れ層：デフォルト（2層：50ノード、100ノード）


In [6]:
from sklearn.preprocessing import StandardScaler

house_data = pd.read_csv('/content/drive/My Drive/DIC/dataset/House_Prices/house_prices_train.csv')

#GrLivArea、YearBuilt、SalePriceを抽出
train = house_data.loc[:, ['GrLivArea', 'YearBuilt', 'SalePrice']]

# DataFrameをndarrayに変換
X = np.array(train.iloc[:, :-1])
y = np.array(train.iloc[:, -1])[:,None]

# 特徴量を標準化
scaler = StandardScaler()
X = scaler.fit_transform(X)
# 目的変数を対数変換
y = np.log1p(y)

# trainとtestに分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
# さらにtrainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:
"""
TensorFlowで実装したニューラルネットワークを使いHousePricesデータセットを多値分類する
"""
tf.compat.v1.disable_eager_execution()

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 1
# 計算グラフに渡す引数の形を決める
X = tf.compat.v1.placeholder("float", [None, n_input])
Y = tf.compat.v1.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([n_input, n_hidden1])),
        'w2': tf.Variable(tf.random_normal([n_hidden1, n_hidden2])),
        'w3': tf.Variable(tf.random_normal([n_hidden2, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([n_hidden1])),
        'b2': tf.Variable(tf.random_normal([n_hidden2])),
        'b3': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.add(tf.matmul(x, weights['w1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
    layer_2 = tf.nn.relu(layer_2)
    layer_output = tf.matmul(layer_2, weights['w3']) + biases['b3'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.squared_difference(Y, logits))
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果（不要）
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.sigmoid(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.squared_difference(Y, logits))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            sess.run(train_op, feed_dict={X: mini_batch_x, Y: mini_batch_y})
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, MSE : {:.3f}, val_MSE : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_MSE : {:.3f}".format(test_acc))

## 変更箇所



# 【問題5】MNISTのモデルを作成

In [ ]:
#《データセットをダウンロードするコード》
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = X_train.astype(np.float)
X_test = X_test.astype(np.float)
X_train /= 255
X_test /= 255

X_train = X_train[:,:,:,None]  # NHWC
X_test = X_test[:,:,:,None]  # NHWC

enc = OneHotEncoder(handle_unknown='ignore', sparse=False)
y_train = enc.fit_transform(y_train[:, np.newaxis])
y_test = enc.transform(y_test[:, np.newaxis])

# trainとvalに分割
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=0)

In [ ]:
"""
TensorFlowで実装したニューラルネットワークを使いMNISTデータセットを多値分類する
"""

# ハイパーパラメータの設定
learning_rate = 0.01
batch_size = 10
num_epochs = 10
n_hidden1 = 50
n_hidden2 = 100
n_input = X_train.shape[1]
n_samples = X_train.shape[0]
n_classes = 10
stride = 1
pad = 'VALID'
ksize = [3, 3]
# 計算グラフに渡す引数の形を決める
X = tf.placeholder("float", [None, X_train.shape[1], X_train.shape[2], X_train.shape[3]])
Y = tf.placeholder("float", [None, n_classes])
# trainのミニバッチイテレータ
get_mini_batch_train = GetMiniBatch(X_train, y_train, batch_size=batch_size)
def example_net(x):
    """
    単純な3層ニューラルネットワーク
    """
    # 重みとバイアスの宣言
    weights = {
        'w1': tf.Variable(tf.random_normal([5,5,1,4])),  # HWCF
        'w2': tf.Variable(tf.random_normal([3,3,4,16])),
        'w3': tf.Variable(tf.random_normal([64, 32])),
        'w4': tf.Variable(tf.random_normal([32, n_classes]))
    }
    biases = {
        'b1': tf.Variable(tf.random_normal([1,1,1,4])),
        'b2': tf.Variable(tf.random_normal([1,1,1,16])),
        'b3': tf.Variable(tf.random_normal([32])),
        'b4': tf.Variable(tf.random_normal([n_classes]))
    }
    layer_1 = tf.nn.conv2d(x, weights['w1'], stride, pad) + biases['b1']
    layer_1 = tf.nn.max_pool2d(layer_1, ksize, ksize, pad)
    layer_2 = tf.nn.conv2d(layer_1, weights['w2'], stride, pad) + biases['b2']
    layer_2 = tf.nn.max_pool2d(layer_2, ksize, ksize, pad)
    layer_2 = tf.layers.Flatten()(layer_2)
    layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
    layer_3 = tf.nn.relu(layer_3)
    layer_output = tf.matmul(layer_3, weights['w4']) + biases['b4'] # tf.addと+は等価である
    return layer_output
# ネットワーク構造の読み込み                               
logits = example_net(X)
# 目的関数
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=logits), axis=0)
# 最適化手法
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)
# 推定結果
correct_pred = tf.equal(tf.argmax(Y, axis=1), tf.argmax(tf.nn.softmax(logits), axis=1))
# correct_pred = tf.equal(tf.sign(Y - 0.5), tf.sign(tf.nn.softmax(logits) - 0.5))
# 指標値計算
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
# variableの初期化
init = tf.global_variables_initializer()

# 計算グラフの実行
with tf.Session() as sess:
    sess.run(init)
    for epoch in range(num_epochs):
        # エポックごとにループ
        total_batch = np.ceil(X_train.shape[0]/batch_size).astype(np.int)
        total_loss = 0
        total_acc = 0
        for i, (mini_batch_x, mini_batch_y) in enumerate(get_mini_batch_train):
            # ミニバッチごとにループ
            _, loss, acc = sess.run([train_op, loss_op, accuracy], feed_dict={X: mini_batch_x, Y: mini_batch_y})
            total_loss += loss
            total_acc += acc
        total_loss /= n_samples
        total_acc /= n_samples
        val_loss, val_acc = sess.run([loss_op, accuracy], feed_dict={X: X_val, Y: y_val})
        print("Epoch {}, loss : {:.4f}, val_loss : {:.4f}, acc : {:.3f}, val_acc : {:.3f}".format(epoch, loss, val_loss, acc, val_acc))
    test_acc = sess.run(accuracy, feed_dict={X: X_test, Y: y_test})
    print("test_acc : {:.3f}".format(test_acc))
    print(sess.run(tf.argmax(tf.nn.softmax(logits), 1), feed_dict={X: X_val, Y: y_val}))

In [ ]:
np.argmax(y_test, axis=1)